In [1]:
dataset = ''
#dataset1 = dataset
from pathlib import Path
output_root = Path("result")
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]

In [2]:
# Parameters
dataset = "CAS"


In [3]:
import pandas as pd
from lenskit import util
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lenskit.datasets import MovieLens
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
from scipy.stats import friedmanchisquare, wilcoxon
from itertools import combinations
from lenskit import util

In [4]:
output_root = Path("result")

In [5]:
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]

In [6]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 1000)

In [7]:
movies = pd.read_csv(r'C:\Users\srabantiguha\lk-demo-experiment\data\ml-20m\movies.csv')
movies = movies[['movieId','genres']]
movies = movies.join(movies.pop('genres').str.get_dummies('|'))
movies.rename({'movieId': 'item'}, axis=1, inplace=True)
movies

,item,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27273,131254,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27274,131256,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27275,131258,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27276,131260,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
dirs = [fld for fld in output_root.glob(f'{dataset}-*')]
dirs

[WindowsPath('result/CAS-ALS'),
 WindowsPath('result/CAS-Bias'),
 WindowsPath('result/CAS-IALS'),
 WindowsPath('result/CAS-IIE'),
 WindowsPath('result/CAS-III'),
 WindowsPath('result/CAS-Pop'),
 WindowsPath('result/CAS-UUE')]

In [9]:
def calc_new_col(row):
    
    if row <= a :
        return 'underexposed'
    
    elif row>= b:
        return 'overexposed'
    
    else:
        return 'Neutral'

In [10]:
analy = pd.DataFrame(columns = ["dataset","shape","Dist_EEL", "quantile(0.25)","quantile(0.75)","accuracy"], index= range(len(dirs)))

for p in range(len(dirs)): 
    
    print('Dataset:-',dirs[p])
    o = dirs[p]
    df = pd.read_csv(dirs[p], sep=',')
    #print(df.shape)

    df = df[['item','difference']]
#     print('Distribution of EEL')
    print(df['difference'].describe())
    print('-----')
    a = df.difference.quantile(0.25) 
    b = df.difference.quantile(0.75) 
#     print("a and b")
#     print(a,b)

    df["category"] = df['difference'].apply(lambda x: calc_new_col(x))

    movies1 = movies.merge(df, on = 'item', how = 'inner')
    map1 = {'underexposed':-1, 'overexposed':1, 'Neutral':0}
    movies1['category'] = movies1['category'].map(map1)
    movies1 = movies1.drop(columns=['item', 'difference'])

    X = movies1.drop(columns=['category'])
    y = movies1['category']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)
    
    
    rf = RandomForestClassifier(random_state=42, n_jobs=-1)
    params = {
        'max_depth': [2,3,5,10,20],
        'min_samples_leaf': [5,10,20,50,100,200],
        'n_estimators': [10,25,30,50,100,200]
    }
    grid_search = GridSearchCV(estimator=rf,param_grid=params,cv = 4,n_jobs=-1, verbose=1, scoring="accuracy")
    grid_search.fit(X_train, y_train)
    y_pred_test = grid_search.predict(X_test)
    ac = (accuracy_score(y_test, y_pred_test))
    
    analy.iloc[p,0] = o
    analy.iloc[p,1] = (df.shape)
    analy.iloc[p,2] = 'Not showed'
    #analy.iloc[p,2] = df['difference'].describe()
    analy.iloc[p,3] = a
    analy.iloc[p,4] = b
    analy.iloc[p,5] = ac
    
analy1 = analy.T
analy1.columns = analy1.iloc[0]
analy1 =  analy1.drop(analy1.index[0])

Dataset:- result\CAS-ALS
count    1.190000e+03
mean     7.966288e+09
std      9.230156e+11
min     -1.810078e+13
25%     -4.928516e+01
50%     -3.196875e+00
75%     -2.665479e-01
max      2.614171e+13
Name: difference, dtype: float64
-----
Fitting 4 folds for each of 180 candidates, totalling 720 fits


Dataset:- result\CAS-Bias
count       476.000000
mean       -298.598230
std        4736.824149
min     -101426.980973
25%         -68.133398
50%         -16.450982
75%          -3.775588
max        8799.077111
Name: difference, dtype: float64
-----
Fitting 4 folds for each of 180 candidates, totalling 720 fits


Dataset:- result\CAS-IALS
count    7346.000000
mean      -13.639407
std        38.404124
min      -656.909994
25%       -10.926433
50%        -3.702359
75%        -1.332031
max       148.848569
Name: difference, dtype: float64
-----
Fitting 4 folds for each of 180 candidates, totalling 720 fits


Dataset:- result\CAS-IIE
count    9.932000e+03
mean     3.494472e+09
std      2.766974e+12
min     -1.435514e+14
25%     -2.449063e+02
50%     -5.576739e+00
75%      2.129433e+02
max      1.880489e+14
Name: difference, dtype: float64
-----
Fitting 4 folds for each of 180 candidates, totalling 720 fits


Dataset:- result\CAS-III
count    3.559000e+03
mean    -2.076707e+38
std      1.311807e+40
min     -7.813921e+41
25%     -2.184531e+01
50%     -5.059494e+00
75%      2.263377e+02
max      4.303694e+40
Name: difference, dtype: float64
-----
Fitting 4 folds for each of 180 candidates, totalling 720 fits


Dataset:- result\CAS-Pop
count     6.390000e+02
mean               NaN
std                NaN
min               -inf
25%     -3.097241e+255
50%       2.317764e+36
75%      7.471486e+255
max                inf
Name: difference, dtype: float64
-----
Fitting 4 folds for each of 180 candidates, totalling 720 fits


Dataset:- result\CAS-UUE
count    7.710000e+02
mean    -4.655656e+05
std      1.358124e+07
min     -2.461037e+08
25%     -1.771607e+02
50%     -7.406041e+00
75%     -2.644789e-01
max      1.488439e+08
Name: difference, dtype: float64
-----
Fitting 4 folds for each of 180 candidates, totalling 720 fits


In [11]:
analy1

dataset,result\CAS-ALS,result\CAS-Bias,result\CAS-IALS,result\CAS-IIE,result\CAS-III,result\CAS-Pop,result\CAS-UUE
shape,"(1190, 3)","(476, 3)","(7346, 3)","(9932, 3)","(3559, 3)","(1409, 3)","(771, 3)"
Dist_EEL,Not showed,Not showed,Not showed,Not showed,Not showed,Not showed,Not showed
quantile(0.25),-49.285156,-68.133398,-10.926433,-244.906272,-21.845312,-3097241463297307844533129600718205940669481104...,-177.160715
quantile(0.75),-0.266548,-3.775588,-1.332031,212.943343,226.337698,7471485643954709916956074872105404260943627769...,-0.264479
accuracy,0.54698,0.445378,0.504627,0.520741,0.482022,0.750708,0.606218


In [12]:
ml20m = MovieLens(r'C:/Users/srabantiguha/lk-demo-experiment/data/ml-20m')
tag_genome = ml20m.tag_genome
tag_genome

tag,007,007 (series),18th century,1920s,1930s,1950s,1960s,1970s,1980s,19th century,...,world politics,world war i,world war ii,writer's life,writers,writing,wuxia,wwii,zombie,zombies
item,,,,,,,,,,,,,,,,,,,,,
1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,...,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,...,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,...,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,...,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,...,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130578,0.15400,0.04025,0.01975,0.01850,0.02850,0.02275,0.01000,0.03675,0.01025,0.01575,...,0.02875,0.01125,0.01250,0.02050,0.21000,0.02375,0.04225,0.00525,0.07575,0.01325
130840,0.02400,0.03025,0.02425,0.09375,0.05600,0.06025,0.02100,0.12900,0.11275,0.03650,...,0.02125,0.04250,0.01025,0.20350,0.47675,0.10775,0.06200,0.00775,0.27000,0.13375
131013,0.04200,0.03175,0.01700,0.05600,0.05750,0.02675,0.01600,0.04200,0.02800,0.03725,...,0.06675,0.01300,0.01275,0.02275,0.20000,0.10750,0.02500,0.00950,0.07550,0.01625


In [13]:
ind = tag_genome.index
ind1 = pd.DataFrame(ind.T, columns=['item'])

In [14]:
tag_genome1 = tag_genome.reset_index(level=[0], drop=True)

In [15]:
tag_genome1 = tag_genome1.iloc[:,1:]

In [16]:
tag_genome1 = pd.concat([ind1,tag_genome1], axis=1)

In [17]:
analyy = pd.DataFrame(columns = ["dataset","shape","Dist_EEL", "quantile(0.25)","quantile(0.75)","accuracy"], index= range(len(dirs)))

for p in range(len(dirs)): 
    
#     print('Dataset:-',dirs[p])
    o = dirs[p]
    df = pd.read_csv(dirs[p], sep=',')
    #print(df.head(2))

    df = df[['item','difference']]
#     print('Distribution of EEL')
#     print(df['difference'].describe())
#     print('-----')

    a = df.difference.quantile(0.25) 
    b = df.difference.quantile(0.75) 
#     print("a and b")
#     print(a,b)

    df["category"] = df['difference'].apply(lambda x: calc_new_col(x))

    movies1 = tag_genome1.merge(df, on = 'item', how = 'inner')
   # print(movies1)
    map1 = {'underexposed':-1, 'overexposed':1, 'Neutral':0}
    movies1['category'] = movies1['category'].map(map1)
    movies1 = movies1.drop(columns=['item', 'difference'])

    X = movies1.drop(columns=['category'])
    y = movies1['category']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)
 
    rf = RandomForestClassifier(random_state=42, n_jobs=-1)
    params = {
        'max_depth': [2,3,5,10,20],
        'min_samples_leaf': [5,10,20,50,100,200],
        'n_estimators': [10,25,30,50,100,200]
    }
    grid_search = GridSearchCV(estimator=rf,param_grid=params,cv = 4,n_jobs=-1, verbose=1, scoring="accuracy")
    grid_search.fit(X_train, y_train)
    y_pred_test = grid_search.predict(X_test)
    ac = (accuracy_score(y_test, y_pred_test))
    
    
    analyy.iloc[p,0] = o
    analyy.iloc[p,1] = (df.shape)
    analyy.iloc[p,2] = 'Not showed'
    #analy.iloc[p,2] = df['difference'].describe()
    analyy.iloc[p,3] = a
    analyy.iloc[p,4] = b
    analyy.iloc[p,5] = ac
    
analy11 = analyy.T
analy11.columns = analy11.iloc[0]
analy11 =  analy11.drop(analy11.index[0])

Fitting 4 folds for each of 180 candidates, totalling 720 fits


Fitting 4 folds for each of 180 candidates, totalling 720 fits


Fitting 4 folds for each of 180 candidates, totalling 720 fits


Fitting 4 folds for each of 180 candidates, totalling 720 fits


Fitting 4 folds for each of 180 candidates, totalling 720 fits


Fitting 4 folds for each of 180 candidates, totalling 720 fits


Fitting 4 folds for each of 180 candidates, totalling 720 fits


In [18]:
analy11

dataset,result\CAS-ALS,result\CAS-Bias,result\CAS-IALS,result\CAS-IIE,result\CAS-III,result\CAS-Pop,result\CAS-UUE
shape,"(1190, 3)","(476, 3)","(7346, 3)","(9932, 3)","(3559, 3)","(1409, 3)","(771, 3)"
Dist_EEL,Not showed,Not showed,Not showed,Not showed,Not showed,Not showed,Not showed
quantile(0.25),-49.285156,-68.133398,-10.926433,-244.906272,-21.845312,-3097241463297307844533129600718205940669481104...,-177.160715
quantile(0.75),-0.266548,-3.775588,-1.332031,212.943343,226.337698,7471485643954709916956074872105404260943627769...,-0.264479
accuracy,0.756477,0.625,0.628035,0.53222,0.64868,0.764205,0.611765
